# 1 Getting VizDoom up and running

In [ ]:
%pip install numpy
%pip install vizdoom

In [2]:
import importlib
import helper
importlib.reload(helper)

<module 'helper' from 'd:\\Desktop\\WIts\\Masters\\Year 1\\Code\\VizDoom\\helper.py'>

In [3]:
import time
import random
import numpy as np
import vizdoom as vd 
from typing import Any
from helper import rgb_to_greyscale, display_grey_scale

In [28]:
game = vd.DoomGame()
game.load_config('github/ViZDoom/scenarios/basic.cfg')
game.init()

In [15]:
actions: np.ndarray = np.identity(3, dtype=np.uint8)

In [ ]:
game = vd.DoomGame()
game.load_config('github/ViZDoom/scenarios/basic.cfg')
game.init()

episodes: int = 10
for episode in range(episodes):
    game.new_episode()

    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        info = state.game_variables
        
        reward = game.make_action(random.choice(actions), 4)
        time.sleep(0.02)

    print("total reward", game.get_total_reward())
    time.sleep(2)
print("done with all")
game.close()

# 2. Converting it to a gym environment

In [ ]:
%pip install gymnasium
%pip install opencv-python

In [5]:
from gymnasium import Env
from gymnasium.spaces import Box, Discrete
import cv2

In [32]:
class VizDoomEnv(Env):
    def __init__(self, render: bool = False) -> None:
        super().__init__()

        self.game = vd.DoomGame()
        self.game.load_config('github/ViZDoom/scenarios/basic.cfg')
        self.game.set_window_visible(render)
        self.game.init()

        self.observation_space: Box = Box(low=1, high=255, shape=(3, 240, 320), dtype=np.uint8)
        self.action_space: Discrete = Discrete(3)
        self.actions: np.ndarray = np.identity(n=3, dtype=np.uint8)

    def close(self) -> None:
        self.game.close()

    # Info should come back as a dict, I added the `| int` because this env is stupid. Doesn't return as a dict for some reason
    def step(self, action) -> tuple[np.ndarray, float, dict[str, Any] | int, bool]:
        reward: float = self.game.make_action(self.actions[action], 4)
        observation = self.get_observation()
        info = self.get_info()
        done = self.game.is_episode_finished()

        return observation, reward, info, done
    
    def reset(self) -> tuple[np.ndarray, dict[str, Any] | int]:
        self.game.new_episode()

        return self.get_observation(), self.get_info()

    def get_observation(self) -> np.ndarray:
        return getattr(self.game.get_state(), "screen_buffer", np.ones_like(self.observation_space.shape, dtype=self.observation_space.dtype))
    
    def get_info(self) -> dict[str, Any] | int:
        return getattr(self.game.get_state(), "game_variables", 0)
    
    def preprocess_observation(self) -> np.ndarray:
        grey_observation: np.ndarray = rgb_to_greyscale(self.get_observation())
        return grey_observation[0:207, :]